In [15]:
# https://github.com/igormq/ctc_tensorflow_example/blob/master/ctc_tensorflow_example.py

#  Compatibility imports
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import tensorflow as tf
import scipy.io.wavfile as wav
import numpy as np
import glob

from six.moves import xrange as range

import soundfile as sf

In [2]:
try:
    from python_speech_features import mfcc
except ImportError:
    print("Failed to import python_speech_features.\n Try pip install python_speech_features.")
    raise ImportError

from utils import maybe_download as maybe_download
from utils import sparse_tuple_from as sparse_tuple_from

In [3]:
# Constants
SPACE_TOKEN = '<space>'
SPACE_INDEX = 0
FIRST_INDEX = ord('a') - 1  # 0 is reserved to space

# Some configs
num_features = 13
num_units=50 # Number of units in the LSTM cell
# Accounting the 0th indice +  space + blank label = 28 characters
num_classes = ord('z') - ord('a') + 1 + 1 + 1

# Hyper-parameters
num_epochs = 10#200
num_hidden = 50
num_layers = 1
batch_size = 1
initial_learning_rate = 1e-2
momentum = 0.9

num_examples = 1
num_batches_per_epoch = int(num_examples/batch_size)

In [4]:
ROOT_DIRECTORY = 'TIMIT_full/train/'

WAV_CLASS = '*.wav'
TXT_CLASS = '*.txt'


In [13]:
def load_dataset(directory_path):

    print(directory_path)
    for e, f in enumerate(glob.glob(directory_path)) :
        
        
        audio, fs = sf.read(f)
#         fs, audio = wav.read(str(f))
        inputs = mfcc(audio, samplerate=fs)
        
        # Tranform in 3D array
        train_inputs = np.asarray(inputs[np.newaxis, :])
        train_inputs = (train_inputs - np.mean(train_inputs))/np.std(train_inputs)
        train_seq_len = [train_inputs.shape[1]]
        
        print('E:',e, ',Len:', train_seq_len, ',FileName:',f)
        

In [14]:
load_dataset(ROOT_DIRECTORY+WAV_CLASS)   

TIMIT_full/train/*/*/*.wav
fs 16000
audio [ 1.52587891e-04  9.15527344e-05  3.05175781e-05 ...  6.10351562e-05
 -6.10351562e-05  9.15527344e-05]
E: 0 ,Len: [272] ,FileName: TIMIT_full/train/dr6/mmdb0/sa1.wav
norm [[[-0.4133509  -1.39542963  0.37596489 ...  0.53152877  0.45852608
    0.24736412]
  [-0.41983402 -1.38390244  0.37551492 ...  0.04018824  0.54333392
    0.53107382]
  [-0.41645274 -1.44844028  0.53282634 ...  0.5154551   0.36089518
    0.9546936 ]
  ...
  [-0.41028475 -1.39298503  0.46951239 ...  0.36741592  0.51100769
    0.98108809]
  [-0.408133   -1.49983445  0.37692412 ... -0.30854691  0.49991754
    0.70151357]
  [-0.41377898 -1.42517416  0.34546041 ...  0.44810956  0.84510248
    0.48243096]]]


In [7]:
# Loading the data

# audio_filename = maybe_download('LDC93S1.wav', 93638)
# target_filename = maybe_download('LDC93S1.txt', 62)

audio_filename = 'sa2.wav'
target_filename ='sa2.txt'

fs, audio = wav.read(audio_filename)

inputs = mfcc(audio, samplerate=fs)
# Tranform in 3D array
train_inputs = np.asarray(inputs[np.newaxis, :])
train_inputs = (train_inputs - np.mean(train_inputs))/np.std(train_inputs)
train_seq_len = [train_inputs.shape[1]]

In [59]:
# Readings targets
with open(target_filename, 'r') as f:

    #Only the last line is necessary
    line = f.readlines()[-1]

    # Get only the words between [a-z] and replace period for none
    original = ' '.join(line.strip().lower().split(' ')[2:]).replace('.', '')
    targets = original.replace(' ', '  ')
    targets = targets.split(' ')

In [60]:
targets

["don't",
 '',
 'ask',
 '',
 'me',
 '',
 'to',
 '',
 'carry',
 '',
 'an',
 '',
 'oily',
 '',
 'rag',
 '',
 'like',
 '',
 'that']

In [61]:
# Adding blank label
targets = np.hstack([SPACE_TOKEN if x == '' else list(x) for x in targets])
print('targets:', targets)

# Transform char into index
targets = np.asarray([SPACE_INDEX if x == SPACE_TOKEN else ord(x) - FIRST_INDEX
                      for x in targets])
print('targets:', targets)

# Creating sparse representation to feed the placeholder
train_targets = sparse_tuple_from([targets])

targets: ['d' 'o' 'n' "'" 't' '<space>' 'a' 's' 'k' '<space>' 'm' 'e' '<space>' 't'
 'o' '<space>' 'c' 'a' 'r' 'r' 'y' '<space>' 'a' 'n' '<space>' 'o' 'i' 'l'
 'y' '<space>' 'r' 'a' 'g' '<space>' 'l' 'i' 'k' 'e' '<space>' 't' 'h' 'a'
 't']
targets: [  4  15  14 -57  20   0   1  19  11   0  13   5   0  20  15   0   3   1
  18  18  25   0   1  14   0  15   9  12  25   0  18   1   7   0  12   9
  11   5   0  20   8   1  20]


In [57]:
targets

array([  4,  15,  14, -57,  20,   0,   1,  19,  11,   0,  13,   5,   0,
        20,  15,   0,   3,   1,  18,  18,  25,   0,   1,  14,   0,  15,
         9,  12,  25,   0,  18,   1,   7,   0,  12,   9,  11,   5,   0,
        20,   8,   1,  20])

In [35]:
# We don't have a validation dataset :(
val_inputs, val_targets, val_seq_len = train_inputs, train_targets, \
                                       train_seq_len

In [36]:
# THE MAIN CODE!

graph = tf.Graph()
with graph.as_default():
    # e.g: log filter bank or MFCC features
    # Has size [batch_size, max_stepsize, num_features], but the
    # batch_size and max_stepsize can vary along each step
    inputs = tf.placeholder(tf.float32, [None, None, num_features])

    # Here we use sparse_placeholder that will generate a
    # SparseTensor required by ctc_loss op.
    targets = tf.sparse_placeholder(tf.int32)

    # 1d array of size [batch_size]
    seq_len = tf.placeholder(tf.int32, [None])

    # Defining the cell
    # Can be:
    #   tf.nn.rnn_cell.RNNCell
    #   tf.nn.rnn_cell.GRUCell 
    cells = []
    for _ in range(num_layers):
        cell = tf.contrib.rnn.LSTMCell(num_units)  # Or LSTMCell(num_units)
        cells.append(cell)
    stack = tf.contrib.rnn.MultiRNNCell(cells)

    # The second output is the last state and we will no use that
    outputs, _ = tf.nn.dynamic_rnn(stack, inputs, seq_len, dtype=tf.float32)

    shape = tf.shape(inputs)
    batch_s, max_timesteps = shape[0], shape[1]

    # Reshaping to apply the same weights over the timesteps
    outputs = tf.reshape(outputs, [-1, num_hidden])

    # Truncated normal with mean 0 and stdev=0.1
    # Tip: Try another initialization
    # see https://www.tensorflow.org/versions/r0.9/api_docs/python/contrib.layers.html#initializers
    W = tf.Variable(tf.truncated_normal([num_hidden,
                                         num_classes],
                                        stddev=0.1))
    # Zero initialization
    # Tip: Is tf.zeros_initializer the same?
    b = tf.Variable(tf.constant(0., shape=[num_classes]))

    # Doing the affine projection
    logits = tf.matmul(outputs, W) + b

    # Reshaping back to the original shape
    logits = tf.reshape(logits, [batch_s, -1, num_classes])

    # Time major
    logits = tf.transpose(logits, (1, 0, 2))

    loss = tf.nn.ctc_loss(targets, logits, seq_len)
    cost = tf.reduce_mean(loss)

    optimizer = tf.train.MomentumOptimizer(initial_learning_rate,
                                           0.9).minimize(cost)

    # Option 2: tf.nn.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
    decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, seq_len)

    # Inaccuracy: label error rate
    ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32),
                                          targets))

In [38]:
with tf.Session(graph=graph) as session:
    # Initializate the weights and biases
    tf.global_variables_initializer().run()


    for curr_epoch in range(num_epochs):
        train_cost = train_ler = 0
        start = time.time()

        for batch in range(num_batches_per_epoch):

            feed = {inputs: train_inputs,
                    targets: train_targets,
                    seq_len: train_seq_len}

            batch_cost, _ = session.run([cost, optimizer], feed)
            train_cost += batch_cost*batch_size
            train_ler += session.run(ler, feed_dict=feed)*batch_size

        train_cost /= num_examples
        train_ler /= num_examples

        val_feed = {inputs: val_inputs,
                    targets: val_targets,
                    seq_len: val_seq_len}

        val_cost, val_ler = session.run([cost, ler], feed_dict=val_feed)

        log = "Epoch {}/{}, train_cost = {:.3f}, train_ler = {:.3f}, val_cost = {:.3f}, val_ler = {:.3f}, time = {:.3f}"
        print(log.format(curr_epoch+1, num_epochs, train_cost, train_ler,
                         val_cost, val_ler, time.time() - start))
    # Decoding
    d = session.run(decoded[0], feed_dict=feed)
    str_decoded = ''.join([chr(x) for x in np.asarray(d[1]) + FIRST_INDEX])
    # Replacing blank label to none
    str_decoded = str_decoded.replace(chr(ord('z') + 1), '')
    # Replacing space label to space
    str_decoded = str_decoded.replace(chr(ord('a') - 1), ' ')

    print('Original:\n%s' % original)
    print('Decoded:\n%s' % str_decoded)

Epoch 1/10, train_cost = 675.332, train_ler = 0.810, val_cost = 458.884, val_ler = 0.810, time = 0.970
Epoch 2/10, train_cost = 458.884, train_ler = 0.976, val_cost = 260.556, val_ler = 0.976, time = 0.342
Epoch 3/10, train_cost = 260.556, train_ler = 0.976, val_cost = 349.951, val_ler = 0.976, time = 0.365
Epoch 4/10, train_cost = 349.951, train_ler = 0.762, val_cost = 224.585, val_ler = 0.762, time = 0.353
Epoch 5/10, train_cost = 224.585, train_ler = 0.905, val_cost = 190.334, val_ler = 0.905, time = 0.351
Epoch 6/10, train_cost = 190.334, train_ler = 0.786, val_cost = 188.816, val_ler = 0.786, time = 0.362
Epoch 7/10, train_cost = 188.816, train_ler = 0.929, val_cost = 166.551, val_ler = 0.929, time = 0.317
Epoch 8/10, train_cost = 166.551, train_ler = 0.881, val_cost = 158.680, val_ler = 0.881, time = 0.338
Epoch 9/10, train_cost = 158.680, train_ler = 0.905, val_cost = 143.112, val_ler = 0.905, time = 0.341
Epoch 10/10, train_cost = 143.112, train_ler = 0.786, val_cost = 137.147,